In [1]:
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
nodes_file = r'C:\c2vsimfg1.5\Preprocessor\C2VSimFG_Nodes.dat'

In [5]:
stratigraphy_file = r'C:\C2VSimFG1.5\Preprocessor\C2VSimFG_Stratigraphy.dat'

In [7]:
groundwater_param_file = r'C:\C2VSimFG1.5\Simulation\Groundwater\C2VSimFG_Groundwater1974.dat'

In [9]:
nodes = pd.read_csv(nodes_file, header=None, \
                    names=['NodeID', 'X', 'Y'], skiprows=90, delim_whitespace=True)

C:\Users\betebari\AppData\Local\Temp\ipykernel_23216\1300167886.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  nodes = pd.read_csv(nodes_file, header=None, \


# nodes

In [12]:
stratigraphy = pd.read_csv(stratigraphy_file, header=None, names=['NodeID', 'GSE', 'A1', 'L1', 'A2',\
                                'L2', 'A3', 'L3', 'A4', 'L4'], skiprows=107, delim_whitespace=True)

C:\Users\betebari\AppData\Local\Temp\ipykernel_23216\2355816574.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  stratigraphy = pd.read_csv(stratigraphy_file, header=None, names=['NodeID', 'GSE', 'A1', 'L1', 'A2',\


In [14]:
stratigraphy.describe().to_csv(r'C:\C2VSimFG1.5\QA_QC\stratigraphy.csv')

In [16]:
# Ensure 'NodeID' has the same data type in both DataFrames
nodes['NodeID'] = nodes['NodeID'].astype(str)  # Convert to string
stratigraphy['NodeID'] = stratigraphy['NodeID'].astype(str)  # Convert to string

# Merge the DataFrames
node_stratigraphy = pd.merge(nodes, stratigraphy, on='NodeID')

# Convert object columns to numeric
numeric_columns = ['A1', 'L1', 'A2', 'L2', 'A3', 'L3', 'A4', 'L4']
for col in numeric_columns:
    node_stratigraphy[col] = pd.to_numeric(node_stratigraphy[col], errors='coerce')

# Fill NaN values with 0
node_stratigraphy = node_stratigraphy.fillna(0)

# Recalculate TotalThickness
node_stratigraphy['TotalThickness'] = (
    node_stratigraphy['A1'] + node_stratigraphy['L1'] +
    node_stratigraphy['A2'] + node_stratigraphy['L2'] +
    node_stratigraphy['A3'] + node_stratigraphy['L3'] +
    node_stratigraphy['A4'] + node_stratigraphy['L4']
)

# Verify the data types
print(node_stratigraphy.dtypes)

# Display the first few rows
print(node_stratigraphy.head())

NodeID             object
X                 float64
Y                 float64
GSE               float64
A1                float64
L1                float64
A2                float64
L2                float64
A3                float64
L3                float64
A4                float64
L4                float64
TotalThickness    float64
dtype: object
  NodeID            X            Y     GSE   A1      L1   A2      L2   A3  \
0      1  554210.8184  4498111.367  616.86  0.0  136.39  0.0  122.35  0.0   
1      2  556163.7904  4499563.238  682.02  0.0  200.44  0.0  127.51  0.0   
2      3  557356.8226  4501930.315  701.69  0.0  203.87  0.0  137.82  0.0   
3      4  559132.7039  4503200.652  682.44  0.0  171.15  0.0  145.29  0.0   
4      5  561893.4429  4503238.309  803.87  0.0  287.87  0.0  148.00  0.0   

       L3   A4    L4  TotalThickness  
0  124.57  0.0  50.0          433.31  
1  104.08  0.0  50.0          482.03  
2   82.27  0.0  50.0          473.96  
3   75.99  0.0  50.0         

In [18]:
# Define the column to plot
field = 'TotalThickness'  # Replace with the correct column name
title = 'Total Thickness of C2VSimFG'
file_name = 'ModelThickness.png'

# Ensure the output directory exists
output_dir = 'C:/Users/betebari/Documents/InSAR-Subsidence'
os.makedirs(output_dir, exist_ok=True)

# Find max and min values
max_value = node_stratigraphy[field].max()
min_value = node_stratigraphy[field].min()

# Extract max X and Y coordinates
max_x = node_stratigraphy[node_stratigraphy[field] == max_value]['X'].iloc[0]
max_y = node_stratigraphy[node_stratigraphy[field] == max_value]['Y'].iloc[0]

# Plotting
plt.figure(figsize=(11, 17))
plt.scatter(node_stratigraphy['X'], node_stratigraphy['Y'], s=4, c=node_stratigraphy[field], cmap='viridis')
plt.grid(True)
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)')
plt.title(title)

# Annotate the maximum value
plt.annotate('{:6.2f} ft'.format(max_value), 
             xy=(max_x, max_y),
             xytext=(max_x + 8000, max_y - 50000), 
             arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.3"))

# Colorbar
cbar = plt.colorbar()
cbar.set_label('Thickness in Feet', fontsize=20)

# Save the file
print("Saving PNG file...")
plt.savefig(os.path.join(output_dir, file_name))
plt.close()
print("File saved successfully.")

Saving PNG file...
File saved successfully.


# Visualize C2VSimFG - layering 1 & 2 & 3 & 4 & Aquitards' data

In [21]:
# List of layer columns
layers = ['L1', 'A2', 'L2', 'L3', 'L4']

# Create a PDF file to save plots and results
output_pdf = 'Layer_Analysis_Results.pdf'
with PdfPages(output_pdf) as pdf:
    for layer in layers:
        # Print the layer name
        print(f"Processing layer: {layer}")
        
        # Scatter plot for the layer
        plt.figure(figsize=(8, 6))
        plt.scatter(node_stratigraphy['X'], node_stratigraphy['Y'], s=4, c=node_stratigraphy[layer], cmap='viridis')
        plt.colorbar(label=f'{layer} Thickness')
        plt.title(f'Spatial Distribution of {layer}')
        plt.xlabel('Easting (m)')
        plt.ylabel('Northing (m)')
        plt.grid(True)
        
        # Save the plot to the PDF
        pdf.savefig()
        plt.close()

        # Calculate statistics for the layer
        max_value = node_stratigraphy[layer].max()
        min_value = node_stratigraphy[layer].min()
        mean_value = node_stratigraphy[layer].mean()
        
        # Create a new figure for statistics
        plt.figure(figsize=(8, 6))
        plt.axis('off')  # Turn off axes
        stats_text = (
            f"Layer: {layer}\n\n"
            f"Maximum Value: {max_value:.2f}\n"
            f"Minimum Value: {min_value:.2f}\n"
            f"Mean Value: {mean_value:.2f}\n"
        )
        plt.text(0.1, 0.5, stats_text, fontsize=12, va='center')
        
        # Save the statistics to the PDF
        pdf.savefig()
        plt.close()

        # Add a normalized column for the layer
        node_stratigraphy[f'{layer}_Normalized'] = (node_stratigraphy[layer] - min_value) / (max_value - min_value)
        print(f"Added normalized column for {layer}.\n")
    
    # Add a summary page to the PDF
    plt.figure(figsize=(8, 6))
    plt.axis('off')
    plt.text(0.1, 0.5, "Layer Analysis Completed\n\nResults saved successfully.", fontsize=16, va='center')
    pdf.savefig()
    plt.close()

# Save the updated DataFrame to CSV
node_stratigraphy.to_csv('Processed_Node_Stratigraphy.csv', index=False)
print("Data saved to 'Processed_Node_Stratigraphy.csv'.")
print(f"Results saved to '{output_pdf}'.")

Processing layer: L1
Added normalized column for L1.

Processing layer: A2
Added normalized column for A2.

Processing layer: L2
Added normalized column for L2.

Processing layer: L3
Added normalized column for L3.

Processing layer: L4
Added normalized column for L4.

Data saved to 'Processed_Node_Stratigraphy.csv'.
Results saved to 'Layer_Analysis_Results.pdf'.


# Step (1) Convert C2VSimFG_Groundwater1974.dat (Aquifer Parameter) to csv file

In [72]:
# Input and output file paths
parsed_data_path = r'C://c2vsimfg1.5//Simulation//Groundwater//C2VSimFG_Groundwater1974.dat'
output_file_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Parsed_AquiferParams.csv'

#   ID   ;   Groundwater node number  NodeID
#   PKH  ;   Hydraulic conductivity; [L/T]
#   PS   ;   Specific storage; [1/L]
#   PN   ;   Specific yield; [L/L]
#   PV   ;   Aquitard vertical hydraulic conductivity; [L/T]
#   PL   ;   Aquifer vertical hydraulic conductivity; [L/T]

# Define columns for the output
columns = ['NodeID', 'Layer', 'X', 'Y', 'PKH', 'PS', 'PN', 'PV', 'PL']
output_data = []

# Define line range for processing
start_line = 54959
end_line = 175675

# Lookup dictionary for X and Y coordinates based on NodeID
xy_lookup = node_stratigraphy.set_index('NodeID')[['X', 'Y']].to_dict('index')

# Read and process the input file
with open(parsed_data_path, 'r', newline='') as input_file:
    current_node_id = None
    current_layer = 0
    current_pkh = current_ps = current_pn = current_pv = current_pl = None

    for i, line in enumerate(input_file):
        # Skip lines outside the specified range
        if i < start_line or i > end_line:
            continue

        # Skip comment lines or empty lines
        if line[0] in ['C', '*'] or line.strip() == '':
            continue

        # Split the line into components
        line_list = line.split()

        # Node rows (start with numeric ID and at least 6 fields)
        if len(line_list) >= 6 and line_list[0].isdigit():
            current_node_id = line_list[0]
            current_layer = 1
            current_pkh, current_ps, current_pn, current_pv, current_pl = line_list[1:6]
            xy = xy_lookup.get(current_node_id, {'X': None, 'Y': None})
            x, y = xy['X'], xy['Y']
            output_data.append([current_node_id, current_layer, x, y, current_pkh, current_ps, current_pn, current_pv, current_pl])

        # Continuation rows (exactly 5 fields)
        elif len(line_list) == 5:
            if current_node_id is not None:
                current_layer += 1
                current_pkh, current_ps, current_pn, current_pv, current_pl = line_list
                xy = xy_lookup.get(current_node_id, {'X': None, 'Y': None})
                x, y = xy['X'], xy['Y']
                output_data.append([current_node_id, current_layer, x, y, current_pkh, current_ps, current_pn, current_pv, current_pl])
            else:
                print(f"Skipping malformed line at index {i}: {line.strip()}")
        else:
            print(f"Skipping unrecognized line at index {i}: {line.strip()}")

# Convert processed data to a DataFrame
groundwater_param = pd.DataFrame(output_data, columns=columns)

# Save the DataFrame to a CSV file
groundwater_param.to_csv(output_file_path, index=False)
print(f"Data saved to {output_file_path}")

# Print the first few rows to verify
print(groundwater_param.head())

Data saved to C://Users//betebari//Documents//InSAR-Subsidence//Parsed_AquiferParams.csv
  NodeID  Layer            X            Y          PKH         PS          PN  \
0      1      1  554210.8184  4498111.367  61.00634182  0.110E-04  0.10819630   
1      1      2  554210.8184  4498111.367  41.02325849  0.741E-05  0.13809980   
2      1      3  554210.8184  4498111.367  22.16909207  0.117E-04  0.10220482   
3      1      4  554210.8184  4498111.367  24.85191058  0.801E-05  0.13311384   
4      2      1  556163.7904  4499563.238  48.58233323  0.151E-04  0.09856682   

           PV          PL  
0  0.00000000  3.35477695  
1  0.00000000  2.10683276  
2  0.00000000  1.10271885  
3  0.00000000  1.12947233  
4  0.00000000  2.19999392  


# Step (2) Convert Aquifer Params(Hydraulic conductivity;Specific yield; etc ...) to Pivotted by Layer  Format

In [76]:
# Input and output file paths
parsed_data_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Parsed_AquiferParams.csv'
output_pivot_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Pivoted_AquiferParams.csv'

# Load the parsed data into a DataFrame
groundwater_param = pd.read_csv(parsed_data_path)

# Pivot the data to create columns for each layer
pivoted_data = groundwater_param.pivot(index=['NodeID', 'X', 'Y'], columns='Layer', values=['PKH', 'PS', 'PN', 'PV', 'PL'])

# Flatten the multi-level column index
pivoted_data.columns = [f'{col[0]}_L{col[1]}' for col in pivoted_data.columns]

# Reset the index to make 'ID', 'X', and 'Y' columns
pivoted_data.reset_index(inplace=True)

# Save the pivoted data to a CSV file
pivoted_data.to_csv(output_pivot_path, index=False)
print(f"Pivoted data saved to {output_pivot_path}")

# Print the first few rows to verify
print(pivoted_data.head())

Pivoted data saved to C://Users//betebari//Documents//InSAR-Subsidence//Pivoted_AquiferParams.csv
   NodeID            X            Y     PKH_L1     PKH_L2     PKH_L3  \
0       1  554210.8184  4498111.367  61.006342  41.023258  22.169092   
1       2  556163.7904  4499563.238  48.582333  28.634508  15.974680   
2       3  557356.8226  4501930.315  44.354940  26.761389  15.331797   
3       4  559132.7039  4503200.652  42.221503  28.970161  15.601213   
4       5  561893.4429  4503238.309  32.068517  21.078297  12.326883   

      PKH_L4     PS_L1     PS_L2     PS_L3  ...     PN_L3     PN_L4  PV_L1  \
0  24.851911  0.000011  0.000007  0.000012  ...  0.102205  0.133114    0.0   
1  23.806886  0.000015  0.000013  0.000017  ...  0.086342  0.132544    0.0   
2  22.826615  0.000021  0.000017  0.000022  ...  0.086610  0.125448    0.0   
3  22.419005  0.000026  0.000020  0.000025  ...  0.085654  0.119488    0.0   
4  18.159367  0.000028  0.000022  0.000028  ...  0.085480  0.111950    0.0   



# Step (3) Convert Subsidence Parameters to csv file

In [116]:
# Input and output file paths
input_file_path = r'C://c2vsimfg1.5//Simulation//Groundwater//C2VSimFG_Subsidence.dat'
output_file_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Parsed_Subsidence.csv'

#	ID	;	Groundwater	node number ;or	NodeID											
#	SCE	;	Elastic	storage	coefficient;	[1/L]											
#	SCI	;	Inelastic	storage	coefficient;	[1/L]											
#	DC	;	Interbed	thickness;	[L]												
#	DCMIN;	Minimum	interbed	thickness;	[L]												
#	HC	;	Pre-compaction	hydraulic	head	(use	99999	to	use	initial	heads);	[L]					
#	*Note*	The	above	land	subsidence	parameters	are	only	for	interbed	layers	(i.e.	clay	layers)												

# Initialize variables
columns = ["NodeID", "Layer", "X", "Y", "SCE", "SCI", "DC", "DCMIN", "HC"]
output_data = []
current_node_id = None
current_layer = 0

# Define line range
start_line = 907
end_line = 121623

# Initialize variables
columns = ["NodeID", "Layer", "X", "Y", "SCE", "SCI", "DC", "DCMIN", "HC"]
output_data = []
current_node_id = None
current_layer = 0

# Define line range for processing
start_line = 907
end_line = 121623

# Process the input file
with open(input_file_path, "r") as file:
    for i, line in enumerate(file, start=1):  # Line numbers start at 1
        # Skip lines outside the range
        if i < start_line or i > end_line:
            continue

        line = line.rstrip()

        # Skip empty lines
        if not line:
            continue

        # Node row: starts with NodeID
        if line[0].isdigit():
            parts = line.split()
            if len(parts) < 6:  # Check if there are enough fields
                print(f"Skipping malformed node row at line {i}: {line}")
                continue

            current_node_id = parts[0]
            current_layer = 1  # Reset layer counter
            sce, sci, dc, dcmin, hc = parts[1:6]

            # Get X and Y coordinates
            xy = xy_lookup.get(current_node_id, {"X": None, "Y": None})
            output_data.append([
                current_node_id,
                current_layer,
                xy["X"],
                xy["Y"],
                float(sce),
                float(sci),
                float(dc),
                float(dcmin),
                float(hc),
            ])

        # Continuation row: starts with whitespace
        elif line[0].isspace():
            parts = line.split()
            if len(parts) < 5:  # Check if there are enough fields
                print(f"Skipping malformed continuation row at line {i}: {line}")
                continue

            current_layer += 1  # Increment layer
            sce, sci, dc, dcmin, hc = parts[:5]

            # Get X and Y coordinates
            xy = xy_lookup.get(current_node_id, {"X": None, "Y": None})
            output_data.append([
                current_node_id,
                current_layer,
                xy["X"],
                xy["Y"],
                float(sce),
                float(sci),
                float(dc),
                float(dcmin),
                float(hc),
            ])
        else:
            print(f"Skipping unrecognized line at line {i}: {line}")

# Convert to DataFrame
df = pd.DataFrame(output_data, columns=columns)

# Save to CSV
df.to_csv(output_file_path, index=False)
print(f"Data saved to {output_file_path}")

# Display the first few rows for verification
print(df.head())

Skipping unrecognized line at line 907: C-------------------------------------------------------------------------------
Data saved to C://Users//betebari//Documents//InSAR-Subsidence//Parsed_Subsidence.csv
  NodeID  Layer            X            Y           SCE       SCI      DC  \
0      1      1  554210.8184  4498111.367  1.668800e-07  0.000016   93.97   
1      1      2  554210.8184  4498111.367  1.668800e-07  0.000016   49.00   
2      1      3  554210.8184  4498111.367  1.668800e-07  0.000016   49.00   
3      1      4  554210.8184  4498111.367  1.668800e-07  0.000016   24.17   
4      2      1  556163.7904  4499563.238  1.635400e-07  0.000020  149.00   

   DCMIN       HC  
0    1.0  99999.0  
1    1.0  99999.0  
2    1.0  99999.0  
3    1.0  99999.0  
4    1.0  99999.0  


# Step (4) Pivot subsidence data by Layer

In [118]:
# File paths
parsed_data_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Parsed_Subsidence.csv'
output_pivot_path = r'C://Users//betebari//Documents//InSAR-Subsidence//pivoted_Subsidence.csv'

# Load the parsed data into a DataFrame
groundwater_param = pd.read_csv(parsed_data_path)

# Verify the unique values in the 'Layer' column
print("Unique layers in the input data:", groundwater_param['Layer'].unique())

# Pivot the data to create columns for each layer
pivoted_data = groundwater_param.pivot(index=['NodeID', 'X', 'Y'], columns='Layer', values=['SCE', 'SCI', 'DC', 'DCMIN', 'HC'])

# Flatten the multi-level column index
pivoted_data.columns = [f'{col[0]}_L{col[1]}' for col in pivoted_data.columns]

# Reset the index to make 'NodeID', 'X', and 'Y' columns
pivoted_data.reset_index(inplace=True)

# Save the pivoted data to a CSV file
pivoted_data.to_csv(output_pivot_path, index=False)
print(f"Pivoted data saved to {output_pivot_path}")

# Print the first few rows to verify
print(pivoted_data.head())

Unique layers in the input data: [1 2 3 4]
Pivoted data saved to C://Users//betebari//Documents//InSAR-Subsidence//pivoted_Subsidence.csv
   NodeID            X            Y        SCE_L1        SCE_L2        SCE_L3  \
0       1  554210.8184  4498111.367  1.668800e-07  1.668800e-07  1.668800e-07   
1       2  556163.7904  4499563.238  1.635400e-07  1.635400e-07  1.635400e-07   
2       3  557356.8226  4501930.315  1.588900e-07  1.588900e-07  1.588900e-07   
3       4  559132.7039  4503200.652  1.556400e-07  1.556400e-07  1.556400e-07   
4       5  561893.4429  4503238.309  1.530400e-07  1.530400e-07  1.530400e-07   

         SCE_L4    SCI_L1    SCI_L2    SCI_L3  ...  DC_L3  DC_L4  DCMIN_L1  \
0  1.668800e-07  0.000016  0.000016  0.000016  ...   49.0  24.17       1.0   
1  1.635400e-07  0.000020  0.000020  0.000020  ...   49.0  24.18       1.0   
2  1.588900e-07  0.000025  0.000025  0.000025  ...   49.0  26.65       1.0   
3  1.556400e-07  0.000029  0.000029  0.000029  ...   49.0  28.7